In [1]:
import pandas as pd
import numpy as np

data_x_df = pd.read_csv('dataset_X.csv')
data_t_df = pd.read_csv('dataset_T.csv')
data_x_df.head()


,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced
0,7,99,Yes,9,1
1,4,82,No,4,2
2,8,51,Yes,7,2
3,5,52,Yes,5,2
4,7,75,No,8,5


In [2]:
D = 5
total_size = data_x_df.shape[0]
for i in range(total_size):
    data_x_df.iloc[i, 2] = 1 if data_x_df.iloc[i, 2] == 'Yes' else 0
    
data_x = np.array(data_x_df, dtype='float64')
data_t = np.array(data_t_df, dtype='float64')

### 1. Feature selection
(a) Evaluate the RMS error

In [3]:
class Regression:
    def __init__(self, x_train, t_train):
        self.x_train = x_train
        self.t_train = t_train

    def basis_vec(self, x_vec, M):
        phi = np.append([1], x_vec)
        if M >= 2:
            for i in range(D):
                phi = np.append(phi, x_vec[i] * x_vec[i: D])
        if M == 3:
            for i in range(D):
                for j in range(i, D):
                    phi = np.append(phi, x_vec[i] * x_vec[j] * x_vec[j: D])
            
        return phi

    def get_w(self, M, lambda_ = 0):
        self.M = M
        phi_mat = np.array([self.basis_vec(xn, M) for xn in self.x_train])
        n = np.size(phi_mat, 1)
        self.w = np.dot(np.dot(np.linalg.pinv(np.dot(phi_mat.T, phi_mat) + np.eye(n)*lambda_), phi_mat.T), self.t_train)
        return self
        
    def RMS_error(self, x, t):
        sum = 0
        n = np.size(x, 0)
        for i in range(n):
            sum += (np.dot(self.w.T, self.basis_vec(x[i], self.M)) - t[i])[0] ** 2
        return np.sqrt(sum / n)

For M = 1

In [4]:
N = int(total_size * 0.8)
x_train = data_x[: N]
t_train = data_t[: N]
x_valid = data_x[N:]
t_valid = data_t[N:]

M = 1
reg = Regression(x_train, t_train)
w_M1 = reg.get_w(M).w
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))

RMS error on the training set: 2.0317614508071022
RMS error on the valid set: 2.060869473062396


For M = 2

In [5]:
M = 2
w_M2 = reg.get_w(M).w
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))

RMS error on the training set: 2.0295179009903648
RMS error on the valid set: 2.0648657388094898


Explain:  
1. The values of the RMS error on the training set for M = 1 and M = 2 are all within 2 ~ 2.2, which means that the regression satisfied our target of expection, so we can conclude that the train data is enough for this model, that is, there is no under-fitting problem for this model.

2. Compared the RMS error on the training set and the valid set, the value of RMS set is close to each other no matter the model is applied with M = 1 or M = 2, and even though the values on the valid set are a little bigger than that on the training set, the smaller increase is within our acceptable range. Therefore, we can conclude that the model may be suitable for the training set.

(b) Select the most contributive feature

In [6]:
M = 1
print('Weight:\n', w_M1.T, '\n')
name = ['Hours Studied', 'Previous Scores', 'Extracurricular Activities', 'Sleep Hours', 'Sample Question Papers Practiced']
for i in range(D):
    print('Feature of %s deleted:'%name[i])
    x_tmp_train = np.delete(x_train, i, axis=1)
    x_tmp_valid = np.delete(x_valid, i, axis=1)
    reg = Regression(x_tmp_train, t_train)
    reg.get_w(M)
    print('RMS error on the training set:', reg.RMS_error(x_tmp_train, t_train))
    print('RMS error on the valid set:', reg.RMS_error(x_tmp_valid, t_valid), '\n')

Weight:
 [[-34.01047532   2.85148294   1.01802319   0.63130645   0.47417895
    0.19139737]] 

Feature of Hours Studied deleted:
RMS error on the training set: 7.649717618347436
RMS error on the valid set: 7.706686993409626 

Feature of Previous Scores deleted:
RMS error on the training set: 17.78392667241751
RMS error on the valid set: 17.749047293349722 

Feature of Extracurricular Activities deleted:
RMS error on the training set: 2.0561157122066143
RMS error on the valid set: 2.077918435001256 

Feature of Sleep Hours deleted:
RMS error on the training set: 2.1849757102033
RMS error on the valid set: 2.232053425832941 

Feature of Sample Question Papers Practiced deleted:
RMS error on the training set: 2.1040542669701194
RMS error on the valid set: 2.143491867735507 



Explain:  
1. From the weight calculated above, we can see the max-value among w-vector is the bias term, and the second highest is the first feature, the hours studied. However, since the data haven't been normalized, the values of weight can't necessarily present the contribution for the following score.

2. Hence, I subsequently delete the five features and then calculate the RMS errors of the remain four features resectively.  From the result above, the RMS increase a lot when the feature -- Previous Score is deleted, which means that the prevoius score is the most contributive feature for the polynomial model with M = 1.

3. All the coefficients on w-vector are postive except for the bias term, so we can conclude that there is no negative contribution among the feature for this model.

### 2. Maximum likelihood approach
(a) I choose Polynomial functions as basis functions for regression because all the five features -- hours studied, previous scores, extracurricular activities, sleep hours, and sample question papers practiced are usually positively related to student performance, and polynomial models have this property. Plus, in the previous question, I obtain a small RMS for M = 2 using a polynomial model, which makes me prefer to increase order and try to construct a improved model.

(b) Linear regression model

For M = 3

In [7]:
M = 3
reg = Regression(x_train, t_train)
reg.get_w(M)
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))

RMS error on the training set: 2.0382734812749996
RMS error on the valid set: 2.0851359963301173


Explain:  
1. In the model with M = 3, I add 35 more basis functions than with M = 2 where M denotes the highest order in this model. When M = 2, besides the bias term and all the features, there are the squares of the features and cross-terms derived by multiplying one another. Therefore, when M = 3, I add the cubes of the five features and the remaining cross-terms, the number of which is H(5, 3) = 35, so the total number of the parameters is 1+5+15+35=56.

2. From the results above, the values of RMS errors are small on the training set and both on the valid set, and the difference between the training set and the valid set is small, which means that the regression satisfied our target of expection and there is no under-fitting problem for this model.

(c) N-fold cross-validation

In [8]:
FOLD = 5
def N_fold(M, lambda_ = 0):
    n = total_size
    mean_err = 0
    for i in range(FOLD):
        x_tmp_train = np.delete(data_x, slice(n*i//FOLD, n*(i+1)//FOLD), axis=0)
        t_tmp_train = np.delete(data_t, slice(n*i//FOLD, n*(i+1)//FOLD), axis=0)
        x_tmp_valid = data_x[n*i//FOLD: n*(i+1)//FOLD]
        t_tmp_valid = data_t[n*i//FOLD: n*(i+1)//FOLD]
        reg = Regression(x_tmp_train, t_tmp_train)
        reg.get_w(M, lambda_)
        mean_err += reg.RMS_error(x_tmp_valid, t_tmp_valid) / FOLD
    return mean_err

For M = 1

In [9]:
M = 1
reg.get_w(M)
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))
print('Mean RMS error for %i-fold on the valid set:' %FOLD, N_fold(M))

RMS error on the training set: 2.0317614508071022
RMS error on the valid set: 2.060869473062396
Mean RMS error for 5-fold on the valid set: 2.0382242992941846


For M = 2

In [10]:
M = 2
reg.get_w(M)
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))
print('Mean RMS error for %i-fold on the valid set:' %FOLD, N_fold(M))

RMS error on the training set: 2.0295179009903648
RMS error on the valid set: 2.0648657388094898
Mean RMS error for 5-fold on the valid set: 2.039830244665117


For M = 3

In [11]:
M = 3
reg.get_w(M)
print('RMS error on the training set:', reg.RMS_error(x_train, t_train))
print('RMS error on the valid set:', reg.RMS_error(x_valid, t_valid))
print('Mean RMS error for %i-fold on the valid set:' %FOLD, N_fold(M))

RMS error on the training set: 2.0382734812749996
RMS error on the valid set: 2.0851359963301173
Mean RMS error for 5-fold on the valid set: 2.058998135305097


Explain:  
1. After five-fold cross-validation, the mean RMS error among the five subsets are all smaller than the situation when we just take one set for validation no matter when M = 1, 2, 3. Hence, the last 20% of the data are less suitable as a validation set than the other four subsets. In addtion, N-fold cross-validation helps us judge a suitable model more comprehensively.

2. From the result above, we can find the RMS errors getting smaller on the training set but a little bit higher on the valid set compared with M = 1 and M = 2, which means that the order M = 3 is about to be enough for the data in this model, and if we apply further higher order for the model, the over-fitting problem may be likely to emerge explicitly.

### 3. MAP approach

(a) The difference between maximum likelihood approach and maximum a posteriori approach is whether the probabilty of prior is considered or not. If a prior obeys Gaussian distribution, we can prove that to maximize the posterior is same as to minimize E(w) + λ||w||^2, and the effect of the prior is equivalent to the regularization term -- lambda in error funcions, the function of which is to reduce over-fitting with the order of the model or the number of parameters increasing.

(b) MAP approach with 5-fold validation

For M = 1

In [12]:
M = 1
L = 0.000001
reg = Regression(x_train, t_train)
print('RMS without λ: ', reg.get_w(M).RMS_error(x_valid, t_valid))
print('RMS for ln λ = %s: ' %L, reg.get_w(M, lambda_=L).RMS_error(x_valid, t_valid))

RMS without λ:  2.060869473062396
RMS for ln λ = 1e-06:  2.060869473733219


For M = 2

In [13]:
M = 2
L = 0.000001
print('RMS without λ: ', reg.get_w(M).RMS_error(x_valid, t_valid))
print('RMS for ln λ = %s: ' %L, reg.get_w(M, lambda_=L).RMS_error(x_valid, t_valid))

RMS without λ:  2.0648657388094898
RMS for ln λ = 1e-06:  2.064865744915659


For M = 3

In [14]:
M = 3
L = 1.6
print('RMS without λ:   ', reg.get_w(M).RMS_error(x_valid, t_valid))
print('RMS for λ = %s: ' %L, reg.get_w(M, lambda_=L).RMS_error(x_valid, t_valid))

RMS without λ:    2.0851359963301173
RMS for λ = 1.6:  2.0809972594030444


(c) Compared the results between maximum likelihood approach and maximum a posteriori approach, for M = 3, after adding λ term, I find there is such a lamda that we can obtain the smallest E(w) + λ||w||^2, but for M = 1 and 2, we can not find that λ since no matter how small the λ I assigned, E(w) + λ||w||^2 still increases. Thus, we can conclude that there is no over-fitting problem for M = 1 and 2, and for M = 3, since the RMS errors (with or without λ) are bigger that for M = 1 and 2, it seems to have a trend of over-fitting as the order gets higher for M > 3. Thus, except for the low orders M = 1 or 2, the result will be consistent with my conclusion in (a) with M >= 3.